### Query by Color

#### Using Histogram

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os 
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds, eigs
import requests
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np

In [2]:
# đầu vào là link của folder keyframes và tên của file map-keyframes
def histogram_vector(image_path):
    # read image 
    img_bgr = cv2.imread(image_path)

    # khởi tạo mảng trống để lưu trữ biểu đồ histogram 
    # arr = np.empty((0, 1944), int)
    
    # convert image from BGR to RGB 
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    # chia khung hình thành các khối 3*3 tổng có 9 blocks
    height, width, channels = img_rgb.shape   
    
    h=0; w=0 
    
    # kích thước của mỗi block
    if height % 3 == 0:
        h_chunk = int(height/3)
    else:
        h_chunk = int(height/3) + 1

    if width % 3 == 0:
        w_chunk = int(width/3)
    else:
        w_chunk = int(width/3) + 1
        
    feature_vector = []
    for a in range(1,4):
        h_window = h_chunk*a
        for b in range(1,4):
            frame = img_rgb[h : h_window, w : w_chunk*b , :]
            """
                tìm histogram cho ảnh, [0, 1, 2] là danh sách  các kênh màu mà histogram được tính toán 
                None: giá trị mask 
                [6, 6, 6] là kích thước của histogram, tức là mỗi kênh màu, histogram sẽ tính toán 6 bin 
                [0, 256, 0, 256, 0, 256] đây là phạm vi giá trị của các kênh màu
            """
            hist = cv2.calcHist(frame, [0, 1, 2], None, [6, 6, 6], [0, 256, 0, 256, 0, 256])  
            hist1= hist.flatten()  # flatten the hist to one-dimensinal vector 
            feature_vector += list(hist1)
            w = w_chunk*b
            
        h = h_chunk*a
        w= 0
    
    return feature_vector 

def image_similarity(image_query_path, folder_dataset_path):
    # khởi tạo dataframe để chứa kết quả
    df_results = pd.DataFrame(columns=['image_path', 'histogram_vector', 'score'])
    
    # get link name of image in folder dataset 
    list_img = os.listdir(folder_dataset_path)
    
    # khởi tạo mảng trống để lưu trữ biểu đồ histogram 
    arr = np.empty((0, 1944), int)
    
    arr = np.vstack((arr, histogram_vector(image_query_path)))
    
    for img_path in list_img: 
        # thêm path của ảnh vào df
        df_results.loc[len(df_results)] = [img_path, None, None]
        
        # tính histogram vector
        feature_vector = histogram_vector(os.path.join(folder_dataset_path, img_path))
        
        arr = np.vstack((arr, feature_vector))
        
    final_arr = arr.transpose() #transposing so that i will have all frames in columns i.e M*N dimensional matrix 
    #where M is 1944 and N is number of frames
    
    # hàm csc_matrix là một hàm trong thư viện scipy sử dụng để tạo 1 ma trận trong định dạng Compressed Sparse Column 
    # định dạng CSC là một định dạng lưu trữ ma trận thưa (sparse matrix) trong đó các phần tử khác không được lưu trữ kèm index của chúng
    A = csc_matrix(final_arr, dtype=float)

    # svds là hàm dùng để phân tích giá trị riêng (singular value decomposition - SVD) trên ma trận thưa (sparse matrix), giúp phân tích một ma trận thành các thành phần như giá trị riêng, vector riêng và các giá trị đặc trưng khác 
    # top 63 singular values from 76082 to 508
    # u là ma trận các vector riêng trái 
    # s là một mảng chứa các giá trị riêng 
    # vt là ma trận chứa các vector riêng phải 
    u, s, vt = svds(A, k = 1940)

    # trả về ma trận chuyển vị của vt
    v1_t = vt.transpose()

    # np.diag(s) tạo ra 1 ma trận đường chéo cho mảng s, các giá trị nằm trên đường chéo bằng với s, còn lại bằng 0 
    # các cột trong ma trận vt là các dữ liệu histogram được chiếu lên cơ sở trực giao
    projections = v1_t @ np.diag(s)    
    
    query_arr = projections[0]
    
    for i in range(1, len(projections)): 
        similarity = np.dot(query_arr, projections[i])/( (np.dot(query_arr,query_arr) **.5) * (np.dot(projections[i], projections[i]) ** .5))
        
        df_results.loc[i-1, 'score'] = similarity
        # df_results.loc[i-1, 'histogram_vector'] = projections[i]
        
    # sắp xếp row theo giá trị giảm dần của score     
    df_sorted = df_results.sort_values(by='score', ascending=False)
    
    return df_sorted.head()

In [3]:
image_query_path = r'dataset\64f85a4b1142fce5030d2036.jpg'
folder_dataset_path = r'dataset'

projections = image_similarity(image_query_path, folder_dataset_path)

print(projections)

                        image_path histogram_vector     score
5388  64f85a4b1142fce5030d2036.jpg             None       1.0
2351  64f854941142fce5030d0bde.jpg             None  0.995431
7749  64f85f241142fce5030d2fc6.jpg             None  0.994001
1053  64f852111142fce5030d03a0.jpg             None  0.993623
317   64f850ad1142fce5030cff93.jpg             None  0.993479


#### Using Kmean-Clustering

In [49]:
def kmc_similarity(image_query_path, folder_dataset_path, num_clusters):
    list_img = os.listdir(folder_dataset_path)
    
    # Mảng projections chứa vector histogram của bạn
    arr = np.empty((0, 1944), int)
    
    # khởi tạo dataframe để truy vấn kết quả
    df_results = pd.DataFrame(columns=['image_path', 'histogram_vector'])

    for img_path in list_img: 
        # tính histogram vector
        feature_vector = histogram_vector(os.path.join(folder_dataset_path, img_path))
        
        arr = np.vstack((arr, feature_vector))
        
        # thêm vào dataframe
        df_results.loc[len(df_results)] = [img_path, str(feature_vector)]
        
    final_arr = arr.copy()
    
    # Xác định số lượng cụm

    # Áp dụng K-means clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(final_arr) # of shape (n_samples, n_features)

    # Truy xuất kết quả
    cluster_labels = kmeans.labels_  # Nhãn cụm cho mỗi vector histogram
    cluster_centers = kmeans.cluster_centers_  # Trung tâm của từng cụm

    # tạo vector histogram 
    vector_test = histogram_vector(image_query_path)

    # Tính khoảng cách từ vector đầu vào đến các trung tâm cụm
    distances = np.linalg.norm(vector_test - cluster_centers, axis=1)

    # Xác định cụm cho vector đầu vào
    predicted_cluster = np.argmin(distances)
    
    # lấy ra chỉ mục các vector trong cụm mà vector được phân vào 
    cluster_indices = np.where(cluster_labels == predicted_cluster)[0]

    # lấy vector trong cụm được dự đoán từ final_arr 
    cluster_vectors = final_arr[cluster_indices]

    # tính khoảng cách Euclidean giữa vector đầu vào và các vector trong cụm 3 
    distances = np.linalg.norm(cluster_vectors - vector_test, axis=1)

    # Lấy 5 chỉ số của các vector có khoảng cách nhỏ nhất đến vector đầu vào
    closest_indices = np.argsort(distances)[:5]
    
    # Lấy các hàng tương ứng với các chỉ số trong danh sách
    selected_rows = df_results.iloc[cluster_indices[closest_indices]]

    # In ra các hàng đã lấy
    print(selected_rows.loc[:, 'image_path'])

In [51]:
image_query_path = r'dataset\64f85a4b1142fce5030d2036.jpg'
folder_dataset_path = r'dataset'

kmc_similarity(image_query_path, folder_dataset_path, num_clusters=20)

c:\Users\Kieu Trung Ha\Desktop\barefoot_task_1\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


5388    64f85a4b1142fce5030d2036.jpg
7749    64f85f241142fce5030d2fc6.jpg
317     64f850ad1142fce5030cff93.jpg
2351    64f854941142fce5030d0bde.jpg
899     64f851d01142fce5030d02c4.jpg
Name: image_path, dtype: object


### Query by Feature

#### Using CNN to extract features (ResNet-50)

In [ ]:
import torch 
import torchvision.models as models 
import torchvision.transforms as transforms 
import os 
import numpy as np 

# Tải mô hình ResNet50 đã được huấn luyện trên ImageNet
model = models.resnet50(pretrained=True)
# Bỏ đi lớp fully connected cuối cùng
model = torch.nn.Sequential(*list(model.children())[:-1])

# lấy list ảnh trong dataset 
img_paths = os.listdir("dataset")

# Chuẩn bị dữ liệu đầu vào cho mô hình
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# khởi tạo list chứa các features 
features = []

for img in img_paths: 
    img_path = os.path.join("dataset", img)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (1024, 1024))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    input_img = transform(img).unsqueeze(0)
    
    # trích xuất đặc trưng từ ảnh 
    with torch.no_grad():
        feature = model(input_img)
    
    features.append(feature)
    
# Lưu mảng thành file .npy
file_path = 'resnet50_features.pkl'  # Đường dẫn để lưu file .npy
np.save(file_path, features)

In [43]:
import pickle

# Đường dẫn đến file .pkl
file_path = 'resnet50_features.pkl'

# Đọc dữ liệu từ file .pkl
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Sử dụng dữ liệu
print(data[0])

[0.6904085  0.00236641 0.03795626 ... 0.         0.         0.        ]


In [41]:
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import torch 
import torchvision.models as models 
import torchvision.transforms as transforms 
import os 
import numpy as np 
import cv2
import pandas as pd

def extract_feature(img_path):
    # Tải mô hình ResNet50 đã được huấn luyện trên ImageNet
    model = models.resnet50(pretrained=True)
    # Bỏ đi lớp fully connected cuối cùng
    model = torch.nn.Sequential(*list(model.children())[:-1])

    # Chuẩn bị dữ liệu đầu vào cho mô hình
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image_path = os.path.join("dataset", img_path)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    input_img = transform(img).unsqueeze(0)
    
    # trích xuất đặc trưng từ ảnh 
    with torch.no_grad():
        feature = model(input_img)
        
    return feature

def query_features(features_path, img_query_path):
    # Đọc file .npy
    data = np.load(features_path, allow_pickle=True)

    img_list = os.listdir("dataset")

    # trích xuất đặc trưng của ảnh truy vấn 
    query_feature = extract_feature(img_query_path).numpy().flatten().reshape(1,-1)
    

    # khởi tạo list chứa danh sách score
    scores_list = []
    
    for feature in data: 
        # chuyển đổi từ tensor sang array
        array_feature = feature.numpy().flatten().reshape(1,-1)

        # tính cosine similarity giữa 2 vector
        cosine_sim = cosine_similarity(query_feature, array_feature)
        
        scores_list.append(cosine_sim)
        
    # Tạo từ điển dữ liệu
    data = {
        "image_path": img_list,
        "scores": scores_list
    }
    
    # Khởi tạo DataFrame
    df = pd.DataFrame(data)
    
    # Sắp xếp DataFrame theo cột "scores" giảm dần
    sorted_df = df.sort_values(by="scores", ascending=False)
    
    return sorted_df.head(5)

In [ ]:
query_features("resnet50_features.npy", "64f85a1a1142fce5030d1fb0.jpg")

### Query by Texture

#### Using GLCM and Gabor

In [2]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans

In [3]:
def extract_gabor_features(img_path):
    # đọc ảnh đầu vào
    img_bgr = cv2.imread(img_path)

    # Chuyển đổi ảnh sang mức xám
    gray_image = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Khởi tạo các thông số Gabor
    num_orientations = 8
    sigma = 2
    lambd = 1
    gamma = 0.1

    # Tạo một kernel Gabor
    gabor_kernel = cv2.getGaborKernel((5, 5), sigma, np.pi/4, lambd, gamma, 0, ktype=cv2.CV_32F)

    # Áp dụng kernel Gabor lên ảnh
    filtered_image = cv2.filter2D(gray_image, cv2.CV_8UC3, gabor_kernel)

    return filtered_image

def extract_glcm_features(gray_image):
    # Tính ma trận co-occurrence 
    glcm = graycomatrix(gray_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

    # Tính các đặc trưng từ ma trận co-occurrence
    contrast = graycoprops(glcm, 'contrast')
    dissimilarity = graycoprops(glcm, 'dissimilarity')
    homogeneity = graycoprops(glcm, 'homogeneity')
    energy = graycoprops(glcm, 'energy')
    correlation = graycoprops(glcm, 'correlation')

    # Gộp các đặc trưng thành một vector
    features = np.concatenate([contrast, dissimilarity, homogeneity, energy, correlation])

    return features

In [4]:
# lấy ra list tất cả path của hình ảnh trong folder
list_image_path = os.listdir("dataset")

# khởi tạo mảng đa chiều để lưu trữ ma trận
matrices = np.zeros((8252, 5, 4))

for i, path in enumerate(list_image_path):
    img_test = extract_gabor_features(os.path.join("dataset", path))
    feature = extract_glcm_features(img_test)

    # Lưu trữ ma trận vào mảng
    matrices[i] = feature

# Lưu mảng ma trận vào tệp tin
np.save("glcm_features.npy", matrices)

In [43]:
from scipy.spatial.distance import cosine
import pandas as pd

def query_texture(features_path, img_query_path):
    # Đọc file .npy
    data = np.load(features_path, allow_pickle=True)

    img_list = os.listdir("dataset")

    # trích xuất đặc trưng của ảnh truy vấn 
    img_test = extract_gabor_features(os.path.join("dataset", img_query_path))
    feature_query = extract_glcm_features(img_test).reshape(-1)

    # khởi tạo list chứa danh sách score
    scores_list = []
    
    for feature in data: 
        # chuyển đổi từ tensor sang array
        array_feature = feature.reshape(-1)
        
        # tính cosine similarity giữa 2 vector
        cosine_sim = 1 - cosine(feature_query, array_feature)
        
        scores_list.append(cosine_sim)
        
    # Tạo từ điển dữ liệu
    data = {
        "image_path": img_list,
        "scores": scores_list
    }
    
    # Khởi tạo DataFrame
    df = pd.DataFrame(data)
    
    # Sắp xếp DataFrame theo cột "scores" giảm dần
    sorted_df = df.sort_values(by="scores", ascending=False)
    
    return sorted_df.head(5)

In [44]:
query_texture('glcm_features.npy', '64f85a0a1142fce5030d1f80.jpg')

,image_path,scores
5250,64f85a0a1142fce5030d1f80.jpg,1.000000
2478,64f854d91142fce5030d0cc7.jpg,0.999895
1571,64f8530d1142fce5030d0746.jpg,0.999807
5892,64f85b421142fce5030d2305.jpg,0.999759
5276,64f85a161142fce5030d1fa6.jpg,0.999703


#### Using Local Binary Pattern 

In [54]:
import cv2
import numpy as np
import os
from skimage.feature import local_binary_pattern

# định nghĩa đối số cho hàm local_binary_pattern
METHOD = 'uniform' # tối đa 2 lần thay đổi 2 giá trị 0-1 1-0
radius = 1 # khoảng cách giữa pixel trung tâm và pixel được so sánh
n_points = 8 * radius # xác định số pixel được so sánh

# lấy ra list tất cả path của ảnh trong tập dataset
img_path = os.listdir("dataset")

features_his = []

for path in img_path:
    # đọc ảnh dưới dạng ảnh xám
    img_bgr = cv2.imread(os.path.join("dataset", path))

    # chuyển ảnh sang dạng rgb
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # tạo LBP cho mỗi ảnh dạng gray vừa tạo và thêm vào list
    lbp_img = local_binary_pattern(img_gray, n_points, radius, METHOD)

    # Tạo histogram
    histogram, bins = np.histogram(lbp_img.flatten(), density=True, bins=10, range=[0, 9])

    features_his.append(histogram)

In [51]:
# hàm tính toán giá trị KLD giữa 2 phân phối xác suất
def kullback_leibler_divergence(dist_p, dist_q):
    '''
    Method compare two probability distributions
    and provide a score of similarity
    :param dist_p: target distribution to compare against
    :param dist_q: comparative distribution
    :return: standardised score determining similarity between distributions
    '''
    p = np.asarray(dist_p)
    q = np.asarray(dist_q)
    # provide number of instances where q and p are the same 
    filt = np.logical_and(p != 0, q != 0)
    # p[filt] provides prob of p multipled by prob of q
    # given p == prob of p, given q
    # Sum to gain overall prob / predictive power
    # p÷q is the likelihood ratio in binary and logBase standardises around 0
    return np.sum(p[filt] * np.log2(p[filt] / q[filt]))

In [52]:
def query_texturev2(features_list, img_query_path):
    img_list = os.listdir("dataset")

    # đọc ảnh dưới dạng ảnh xám
    img_bgr = cv2.imread(os.path.join("dataset", img_query_path))

    # chuyển ảnh sang dạng rgb
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # tạo LBP cho mỗi ảnh dạng gray vừa tạo và thêm vào list
    lbp_img = local_binary_pattern(img_gray, n_points, radius, METHOD)

    # Tạo histogram
    feature_query, bins = np.histogram(lbp_img.flatten(), density=True, bins=10, range=[0, 9])

    # khởi tạo list chứa danh sách score
    scores_list = []
    
    for feature in features_list: 
        # chuyển đổi từ tensor sang array
        array_feature = feature.reshape(-1)
        
        # tính cosine similarity giữa 2 vector
        kld_val = kullback_leibler_divergence(feature_query.reshape(-1), array_feature)
        
        scores_list.append(kld_val)
        
    # Tạo từ điển dữ liệu
    data = {
        "image_path": img_list,
        "scores": scores_list
    }
    
    # Khởi tạo DataFrame
    df = pd.DataFrame(data)
    
    # Sắp xếp DataFrame theo cột "scores" giảm dần
    sorted_df = df.sort_values(by="scores", ascending=True)
    
    return sorted_df.head(5)    

In [55]:
query_texturev2(features_his, '64f860391142fce5030d335c.jpg')

,image_path,scores
8238,64f860391142fce5030d335c.jpg,0.000000
7895,64f85f7d1142fce5030d3123.jpg,0.000628
5645,64f85ac31142fce5030d21a3.jpg,0.000716
2130,64f854271142fce5030d0a63.jpg,0.001151
4972,64f8598a1142fce5030d1e15.jpg,0.002260
